In [1]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

In [2]:
messages = Queue()
recordings = Queue()

In [3]:
# record_button = widgets.Button(
#     description="Record",
#     disabled=False,
#     button_style="success",
#     icon="Microphone"
#     )

# stop_button = widgets.Button(
#     description="Stop",
#     disabled=False,
#     button_style="warning",
#     icon="Stop"
#     )

# output = widgets.Output()

# def start_recording(data):
#     messages.put("True")
    
#     with output:
#         display("Starting...")
#         record = Thread(target=record_microphone)
#         record.start()
        
#         transcribe = Thread(target=speech_recognition, args=(output))
        
# def stop_recording(data):
#     with output:
#         messages.get()
#         display("Stopped.")      
        
# record_button.on_click(start_recording)
# stop_button.on_click(stop_recording)





# display(record_button, stop_button, output)

In [4]:
import pyaudio

In [5]:
p =pyaudio.PyAudio()
for i in range (p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 4, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 

In [6]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

In [7]:
def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT,
                     channels=CHANNELS,
                     rate=FRAME_RATE,
                     input=True,
                     input_device_index=1,#14,#15,#16
                     frames_per_buffer=chunk)
    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()

In [8]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames)) # joins all chunks together
        result = rec.Result()
        text = json.loads(result)["text"] # vosk returns a json string, so we need to parse it
        
        cased = subprocess.check_output("python vosk-recasepunc-en-0.22/recasepunc.py predict vosk-recasepunc-en-0.22/checkpoint", shell=True, input=text)
        output.append_stdout(cased)
        

In [9]:
record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="Microphone"
    )

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="Stop"
    )

output = widgets.Output()

def start_recording(data):
    messages.put("True")
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output))
        transcribe.start()
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")      
        
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)





display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='Microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='Stop', style=ButtonStyle())

Output()

    self.run()
  File "C:\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.speech_recognition() argument after * must be an iterable, not Output


In [ ]:
# TO DO:
# check microphone index and maybe with headphones?